# 使用 🤗 Diffusers 实现 Stable Diffusion

**Pedro Cuenca, Patrick von Platen, Suraj Patil, Jeremy Howard**

你可能已经在 Twitter（或其他地方）看到过很多通过输入简短场景描述就能生成图像的例子。这是多年来生成模型 (Generative Model) 研究的成果。本笔记本将介绍 Stable Diffusion —— 目前质量最高的开源文生图模型。它足够小巧，可以在消费级 GPU 上运行，而不需要数据中心级别的算力。我们使用 🤗 Hugging Face 的 [🧨 Diffusers 库](https://github.com/huggingface/diffusers)，这是目前我们推荐用于扩散模型 (Diffusion Model) 的库。

在本课程中，我们将会看到，理解最先进的生成模型需要深入理解现代机器学习模型中的许多基础模块。本笔记本将展示 Stable Diffusion 的能力，并简要介绍其主要组件。

_如果你在 Colab 中打开本笔记本，或者在生成第一张图片时遇到类型错误，请取消注释并运行下面的单元格。_

In [ ]:
!pip install -Uq diffusers transformers fastcore

## 使用 Stable Diffusion

要在你的电脑上运行 Stable Diffusion，你需要接受模型许可协议。这是一个开放的 CreativeML OpenRail-M 许可证，它不对你生成的输出主张任何权利，但禁止你故意生成非法或有害内容。[模型卡片](https://huggingface.co/CompVis/stable-diffusion-v1-4) 提供了更多详情。如果你接受该许可证，你需要成为 🤗 Hugging Face Hub 的注册用户，并使用访问令牌 (Access Token) 才能运行代码。你有两种方式提供访问令牌：

* 在终端中使用 `huggingface-cli login` 命令行工具，并在提示时粘贴你的令牌。令牌会保存在你电脑上的文件中。
* 或者在笔记本中使用 `notebook_login()`，功能相同。

In [ ]:
import logging
from pathlib import Path

import matplotlib.pyplot as plt
import torch
from diffusers import StableDiffusionPipeline
from fastcore.all import concat
from huggingface_hub import notebook_login
from PIL import Image

logging.disable(logging.WARNING)

torch.manual_seed(1)
if not (Path.home()/'.cache/huggingface'/'token').exists(): notebook_login()

### Stable Diffusion 流水线 (Pipeline)

[`StableDiffusionPipeline`](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion#diffusers.StableDiffusionPipeline) 是一个端到端的[扩散推理流水线](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion)，只需几行代码就能开始生成图像。许多 Hugging Face 库（以及其他库如 scikit-learn）都使用"流水线"(Pipeline) 的概念来表示一系列组合在一起完成某项任务的步骤。我们稍后会详细了解流水线的各个步骤 - 现在，让我们先用它来看看它能做什么。

当我们说"推理" (Inference) 时，指的是使用现有模型生成样本（在本例中是图像），而不是使用新数据"训练" (Training) 或微调 (Fine-tuning) 模型。

我们使用 [`from_pretrained`](https://huggingface.co/docs/diffusers/main/en/api/diffusion_pipeline#diffusers.DiffusionPipeline.from_pretrained) 来创建流水线并下载预训练权重。我们指定使用 `fp16`（半精度）版本的权重，并告诉 `diffusers` 期望该格式的权重。这使我们能够以更快的速度进行推理，而质量几乎没有明显差异。传递给 `from_pretrained` 的字符串（本例中为 `CompVis/stable-diffusion-v1-4`）是托管在 [Hugging Face Hub](https://huggingface.co/models) 上的预训练流水线的仓库 ID；它也可以是包含流水线权重的本地目录路径。首次运行此单元格时，流水线中所有模型的权重将被下载并缓存。

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", variant="fp16", torch_dtype=torch.float16).to("cuda")

权重默认缓存在你的主目录中。

In [ ]:
!ls ~/.cache/huggingface/hub

现在我们准备好使用流水线来开始创建图像了。

如果你的 GPU 内存不足以使用 `pipe`，请运行 `pipe.enable_attention_slicing()`

如文档所述：
> 启用此选项后，注意力模块会将输入张量切片，分多个步骤计算注意力。这有助于节省一些内存，但会略微降低速度。

In [ ]:
#pipe.enable_attention_slicing()

In [ ]:
prompt = "a photograph of an astronaut riding a horse"

In [ ]:
pipe(prompt).images[0]

In [ ]:
torch.manual_seed(1024)
pipe(prompt).images[0]

你会注意到运行流水线时会显示一个带有一定步数的进度条。这是因为 Stable Diffusion 基于一种渐进式去噪算法，能够从纯随机噪声开始创建出逼真的图像。这类模型被称为 _扩散模型 (Diffusion Model)_。下面是一个模型绘制手写数字的过程示例（从顶部的随机噪声到底部逐渐改进的图像），我们将在课程后面从零开始构建这样的模型。

![image.png](attachment:image.png)

In [ ]:
torch.manual_seed(1024)
pipe(prompt, num_inference_steps=3).images[0]

In [ ]:
torch.manual_seed(1024)
pipe(prompt, num_inference_steps=16).images[0]

### 分类器自由引导 (Classifier-Free Guidance)

In [ ]:
def image_grid(imgs, rows, cols):
    w,h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    for i, img in enumerate(imgs): grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

_分类器自由引导 (Classifier-Free Guidance)_ 是一种增强输出与我们使用的条件信号（文本）一致性的方法。

简单来说，引导值越大，模型越努力地去表现文本提示词。但是，较大的值往往会产生较少的多样性。默认值是 `7.5`，它代表了多样性和保真度之间的良好平衡。这篇[博客文章](https://benanne.github.io/2022/05/26/guidance.html)更深入地介绍了其工作原理。

我们可以通过简单地传递一个提示词列表而不是字符串来为同一个提示词生成多张图像。

In [ ]:
num_rows,num_cols = 4,4
prompts = [prompt] * num_cols

In [ ]:
images = concat(pipe(prompts, guidance_scale=g).images for g in [1.1,3,7,14])

In [ ]:
image_grid(images, rows=num_rows, cols=num_cols)

### 负面提示词 (Negative Prompts)

_负面提示词 (Negative Prompting)_ 指的是使用另一个提示词（而不是完全无条件的生成），并缩放该提示词生成结果与条件生成结果之间的差异。

In [ ]:
torch.manual_seed(1000)
prompt = "Labrador in the style of Vermeer"
pipe(prompt).images[0]

In [ ]:
torch.manual_seed(1000)
pipe(prompt, negative_prompt="blue").images[0]

通过使用负面提示词，我们更多地朝着正面提示词的方向移动，有效地降低了负面提示词在我们构图中的重要性。

### 图生图 (Image to Image)

虽然 Stable Diffusion 是为生成图像而训练的，并且可以选择性地使用文本条件来引导生成，但我们可以将原始的图像扩散过程用于其他任务。

例如，我们可以不从纯噪声开始，而是从一张图像开始，并向其添加一定量的噪声。我们替换去噪过程的初始步骤，假装我们的图像是算法生成的。然后我们像往常一样从那个状态继续扩散过程。

这通常会保留构图，尽管细节可能会发生很大变化。这对于草图来说非常棒！

这些操作（提供初始图像、向其添加一些噪声并从那里运行扩散）可以由一个专门的图生图流水线自动执行：`StableDiffusionImg2ImgPipeline`。这是其 [`__call__` 方法](https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion_img2img.py#L124) 的源代码。

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline
from fastdownload import FastDownload

In [ ]:
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    variant="fp16",
    torch_dtype=torch.float16,
).to("cuda")

我们将使用以下由 [用户 VigilanteRogue81](https://huggingface.co/spaces/huggingface-projects/diffuse-the-rest/discussions/204) 创建的草图作为示例。

In [ ]:
p = FastDownload().download('https://cdn-uploads.huggingface.co/production/uploads/1664665907257-noauth.png')
init_image = Image.open(p).convert("RGB")
init_image

In [ ]:
torch.manual_seed(1000)
prompt = "Wolf howling at the moon, photorealistic 4K"
images = pipe(prompt=prompt, num_images_per_prompt=3, image=init_image, strength=0.8, num_inference_steps=50).images
image_grid(images, rows=1, cols=3)

当我们得到一个喜欢的构图后，可以将它用作下一个提示词的种子，进一步改变结果。例如，让我们使用上面的第三张图像，尝试用它生成梵高风格的作品。

In [ ]:
init_image = images[2]

In [ ]:
torch.manual_seed(1000)
prompt = "Oil painting of wolf howling at the moon by Van Gogh"
images = pipe(prompt=prompt, num_images_per_prompt=3, image=init_image, strength=1, num_inference_steps=70).images
image_grid(images, rows=1, cols=3)

有创造力的人使用不同的工具进行迭代优化，以实现他们脑海中的想法。这里有一个[工具建议列表](https://github.com/fastai/diffusion-nbs/blob/43a090286e5742f807d4ff58524c02a1888b3004/suggested_tools.md)可以帮助你入门。

### 微调 (Fine-tuning)

[我们如何在 Lambda 制作文本生成宝可梦模型](https://lambdalabs.com/blog/how-to-fine-tune-stable-diffusion-how-we-made-the-text-to-pokemon-model-at-lambda/)

![](https://lambdalabs.com/hs-fs/hubfs/2.%20Images/Images%20-%20Blog%20Posts/2022%20-%20Blog%20Images/image--3-.png?width=1152&height=768&name=image--3-.png)

戴珍珠耳环的少女、可爱的奥巴马生物、唐纳德·特朗普、鲍里斯·约翰逊、龙猫、Hello Kitty

### 文本反演 (Textual Inversion)

文本反演 (Textual Inversion) 是一种可以快速"教会"文本模型一个新词，并训练其嵌入向量接近某种视觉表示的过程。这是通过向词汇表添加一个新 token，冻结所有模型的权重（文本编码器除外），然后用几张代表性图像进行训练来实现的。

这是[论文作者](https://textual-inversion.github.io)对该过程的示意图。

![Textual Inversion 示意图](https://textual-inversion.github.io/static/images/training/training.JPG)

---

你可以使用你提供的照片通过[这个训练脚本](https://github.com/huggingface/diffusers/tree/main/examples/textual_inversion)或 [Google Colab 笔记本](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/sd_textual_inversion_training.ipynb)训练你自己的 token。还有一个[用于推理的 Colab 笔记本](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_conceptualizer_inference.ipynb)，但我们将在下面展示将训练好的 token 添加到词汇表并使其与预训练的 Stable Diffusion 模型一起工作所需遵循的步骤。

我们将使用为[这种风格](https://huggingface.co/sd-concepts-library/indian-watercolor-portraits)训练的嵌入来尝试一个示例。

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", variant="fp16", torch_dtype=torch.float16) 
pipe = pipe.to("cuda")

In [ ]:
embeds_url = "https://huggingface.co/sd-concepts-library/indian-watercolor-portraits/resolve/main/learned_embeds.bin"
embeds_path = FastDownload().download(embeds_url)
embeds_dict = torch.load(str(embeds_path), map_location="cpu")

新 token 的嵌入向量存储在一个小型 PyTorch 序列化字典中，其键是训练的新文本 token。由于我们流水线的编码器不知道这个词，我们需要手动添加它。

In [ ]:
tokenizer = pipe.tokenizer
text_encoder = pipe.text_encoder
new_token, embeds = next(iter(embeds_dict.items()))
embeds = embeds.to(text_encoder.dtype)
new_token

我们将新 token 添加到分词器 (tokenizer)，并将训练好的嵌入向量添加到嵌入表中。

In [ ]:
assert tokenizer.add_tokens(new_token) == 1, "The token already exists!"

In [ ]:
text_encoder.resize_token_embeddings(len(tokenizer))
new_token_id = tokenizer.convert_tokens_to_ids(new_token)
text_encoder.get_input_embeddings().weight.data[new_token_id] = embeds

现在我们可以运行推理，并像引用字典中的另一个单词一样引用这个风格。

In [ ]:
torch.manual_seed(1000)
image = pipe("Woman reading in the style of <watercolor-portrait>").images[0]
image

### Dreambooth

[Dreambooth](https://dreambooth.github.io) 是一种微调方法，它试图通过只提供少量新主题的图像来引入新主题。其目标与[文本反演](#文本反演-textual-inversion)类似，但过程不同。Dreambooth 不像文本反演那样创建一个新 token，而是选择词汇表中一个现有的 token（通常是一个很少使用的），并对模型进行几百步的微调，使该 token 接近我们提供的图像。这是一个常规的微调过程，其中所有模块都被解冻。

例如，我们使用类似 `"photo of a sks person"` 的提示词微调了一个模型，使用罕见的 `sks` token 来限定 `person` 这个词，并使用 Jeremy 的照片作为目标。让我们看看效果如何。

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("pcuenq/jh_dreambooth_1000", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

In [ ]:
torch.manual_seed(1000)

prompt = "Painting of sks person in the style of Paul Signac"
images = pipe(prompt, num_images_per_prompt=4).images
image_grid(images, 1, 4)

使用 Dreambooth 进行微调是比较棘手的，对超参数很敏感，因为我们本质上是在要求模型对提供的图像过拟合提示词。在某些情况下，我们会观察到一些问题，如相关术语（本例中的 `"person"`）的灾难性遗忘。作者应用了一种叫做"先验保持" (Prior Preservation) 的技术来尝试避免这种情况，通过使用该术语的其他示例（除了我们提供的那些）进行特殊正则化。这个想法的酷之处在于，这些示例可以由预训练的 Stable Diffusion 模型本身轻松生成！我们在为上一个示例训练的模型中没有使用该方法。

其他可能有效的想法包括：使用 EMA 使最终权重保留一些先前的知识、使用渐进式学习率进行微调，或者将文本反演的优点与 Dreambooth 结合起来。这些都可以成为一些有趣的项目来尝试！

如果你想训练自己的 Dreambooth 模型，可以使用[这个脚本](https://github.com/huggingface/diffusers/tree/main/examples/dreambooth)或[这个 Colab 笔记本](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/sd_dreambooth_training.ipynb)。

### 什么是 Stable Diffusion

潜在扩散 (Latent Diffusion) 有三个主要组件：

1. 自编码器 (Autoencoder)，即 VAE (变分自编码器，Variational Autoencoder)。
2. [U-Net](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb#scrollTo=wW8o1Wp0zRkq)。
3. 文本编码器，例如 [CLIP 的文本编码器](https://huggingface.co/docs/transformers/model_doc/clip#transformers.CLIPTextModel)。

U-Net 的输出是噪声残差，用于通过调度器算法 (Scheduler Algorithm) 计算去噪后的潜在图像表示。可以使用许多不同的调度器算法进行此计算，每种算法都有其优缺点。对于 Stable Diffusion，我们推荐使用以下之一：

- [PNDM 调度器](https://github.com/huggingface/diffusers/blob/main/src/diffusers/schedulers/scheduling_pndm.py)（默认使用）
- [DDIM 调度器](https://github.com/huggingface/diffusers/blob/main/src/diffusers/schedulers/scheduling_ddim.py)
- [K-LMS 调度器](https://github.com/huggingface/diffusers/blob/main/src/diffusers/schedulers/scheduling_lms_discrete.py)

### 潜变量和回调 (Latents and Callbacks)

Stable Diffusion 基于一种特殊类型的扩散模型，称为**潜在扩散 (Latent Diffusion)**，在论文 [High-Resolution Image Synthesis with Latent Diffusion Models](https://arxiv.org/abs/2112.10752) 中提出。

通用扩散模型是被训练来逐步*去噪*随机高斯噪声的机器学习系统，最终得到我们感兴趣的样本，例如*图像*。有关其工作原理的更详细概述，请查看[这个 Colab](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb)。

扩散模型已被证明在生成图像数据方面达到了最先进的结果。但扩散模型的一个缺点是反向去噪过程很慢。此外，这些模型消耗大量内存，因为它们在像素空间 (Pixel Space) 中操作，当生成高分辨率图像时，这变得非常昂贵。因此，训练这些模型以及将它们用于推理都具有挑战性。

潜在扩散可以通过在较低维度的 _潜空间 (Latent Space)_ 上应用扩散过程来减少内存和计算复杂度，而不是使用实际的像素空间。这是标准扩散和潜在扩散模型之间的关键区别：**在潜在扩散中，模型被训练来生成图像的潜在（压缩的）表示。**

Stable Diffusion 流水线可以将中间潜变量发送到我们提供的回调函数。通过将这些潜变量通过图像解码器（流水线的 `vae` 组件）运行，我们可以看到去噪过程如何进行以及图像如何逐渐展开。

In [ ]:
vae = pipe.vae
images = []

def latents_callback(i, t, latents):
    latents = 1 / 0.18215 * latents
    image = vae.decode(latents).sample[0]
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.cpu().permute(1, 2, 0).numpy()
    images.extend(pipe.numpy_to_pil(image))

prompt = "Portrait painting of Jeremy Howard looking happy."
torch.manual_seed(9000)
final_image = pipe(prompt, callback=latents_callback, callback_steps=12).images[0]
images.append(final_image)
image_grid(images, rows=1, cols=len(images))

**为什么潜在扩散既快速又高效？**

由于潜在扩散模型的 U-Net 在低维空间上操作，与像素空间扩散模型相比，它大大减少了内存和计算需求。例如，Stable Diffusion 中使用的自编码器的缩减因子为 8，但使用 4 个通道而不是 3 个。这意味着形状为 `(3, 512, 512)` 的图像在潜空间中变为 `(4, 64, 64)`，这需要的内存减少了 `8 × 8 × 3/4 = 48` 倍。

这就是为什么即使在 16GB 的 Colab GPU 上也能如此快速地生成 `512 × 512` 图像的原因！

In [ ]:
del pipe

## 深入流水线内部

推理流水线只是一小段代码，将各个组件连接在一起并执行推理循环。[这就是它的全部内容](https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py#L204)。

我们将逐步加载和连接各个部分，看看我们如何自己编写它。我们将首先从预训练权重加载所需的所有模块。

首先，我们需要文本编码器和分词器。这些来自标准 CLIP 模型的文本部分，所以我们将使用 OpenAI 发布的权重。

In [ ]:
from transformers import CLIPTextModel, CLIPTokenizer

In [ ]:
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16)
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16).to("cuda")

接下来我们加载 `vae` 和 `unet`。这些是独立的模型，其权重存储在 Stable Diffusion 仓库的文件夹中。我们可以使用 `subfolder` 参数来引用[这些位置](https://huggingface.co/CompVis/stable-diffusion-v1-4/tree/main)。

In [ ]:
from diffusers import AutoencoderKL, UNet2DConditionModel

In [ ]:
# 这里我们使用与原始版本不同的 VAE，它经过了更多步骤的微调
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-ema", torch_dtype=torch.float16).to("cuda")
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet", torch_dtype=torch.float16).to("cuda")

为了使事情有所不同，我们将使用另一个调度器。标准流水线使用 [PNDM 调度器](https://arxiv.org/abs/2202.09778)，但我们将使用 [Katherine Crowson](https://github.com/crowsonkb) 出色的 K-LMS 调度器。

我们需要小心使用与训练期间相同的噪声计划。该计划由噪声步数和每一步添加的噪声量定义，这是从 _beta_ 参数推导出来的。

对于 K-LMS 调度器，以下是训练期间使用的 1000 步噪声过程中 beta 的演变：

In [ ]:
beta_start,beta_end = 0.00085,0.012
plt.plot(torch.linspace(beta_start**0.5, beta_end**0.5, 1000) ** 2)
plt.xlabel('Timestep')
plt.ylabel('β');

In [ ]:
from diffusers import LMSDiscreteScheduler

In [ ]:
scheduler = LMSDiscreteScheduler(beta_start=beta_start, beta_end=beta_end, beta_schedule="scaled_linear", num_train_timesteps=1000)

现在我们定义用于生成的参数。

与之前的示例不同，我们将 `num_inference_steps` 设置为 70 以获得更清晰的图像。

In [ ]:
prompt = ["a photograph of an astronaut riding a horse"]

height = 512
width = 512
num_inference_steps = 70
guidance_scale = 7.5
batch_size = 1

我们对提示词进行分词。模型要求每个提示词具有相同数量的 token，因此使用填充 (Padding) 来确保我们达到所需的长度。

In [ ]:
text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
text_input['input_ids']

In [ ]:
tokenizer.decode(49407)

注意力掩码 (Attention Mask) 使用零来表示我们不感兴趣的 token。这些都是填充 token。

In [ ]:
text_input['attention_mask']

文本编码器为我们使用的文本提示词提供嵌入向量。

In [ ]:
text_embeddings = text_encoder(text_input.input_ids.to("cuda"))[0].half()
text_embeddings.shape

我们还获取执行无条件生成所需的嵌入向量，这是通过空字符串实现的：模型可以自由地朝任何方向发展，只要它产生看起来合理的图像。这些嵌入向量将用于应用分类器自由引导。

In [ ]:
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
)
uncond_embeddings = text_encoder(uncond_input.input_ids.to("cuda"))[0].half()
uncond_embeddings.shape

对于分类器自由引导，我们需要进行两次前向传播。一次使用条件输入 (`text_embeddings`)，另一次使用无条件嵌入 (`uncond_embeddings`)。在实践中，我们可以将两者连接成一个批次，以避免进行两次前向传播。

In [ ]:
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

要开始去噪过程，我们从纯高斯（正态）噪声开始。这些是我们的初始潜变量。

In [ ]:
torch.manual_seed(100)
latents = torch.randn((batch_size, unet.in_channels, height // 8, width // 8))
latents = latents.to("cuda").half()
latents.shape

`4×64×64` 是输入形状。在去噪过程完成后，解码器稍后会将这个潜在表示转换为 `3×512×512` 的图像。

接下来，我们用选择的 `num_inference_steps` 初始化调度器。这将准备在去噪过程中使用的内部状态。

In [ ]:
scheduler.set_timesteps(num_inference_steps)

我们按调度器要求的标准差缩放初始噪声。这个值将取决于我们使用的特定调度器。

In [ ]:
latents = latents * scheduler.init_noise_sigma

我们准备好编写去噪循环了。时间步从 `999` 到 `0`（训练期间使用的 1000 步）按照特定的计划进行。

In [ ]:
scheduler.timesteps

In [ ]:
scheduler.sigmas

In [ ]:
plt.plot(scheduler.timesteps, scheduler.sigmas[:-1]);

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for i, t in enumerate(tqdm(scheduler.timesteps)):
    input = torch.cat([latents] * 2)
    input = scheduler.scale_model_input(input, t)

    # 预测噪声残差
    with torch.no_grad(): pred = unet(input, t, encoder_hidden_states=text_embeddings).sample

    # 执行引导
    pred_uncond, pred_text = pred.chunk(2)
    pred = pred_uncond + guidance_scale * (pred_text - pred_uncond)

    # 计算"前一个"噪声样本
    latents = scheduler.step(pred, t, latents).prev_sample

这个过程完成后，我们的 `latents` 包含图像的去噪表示。我们使用 `vae` 解码器将其转换回像素空间。

In [ ]:
with torch.no_grad(): image = vae.decode(1 / 0.18215 * latents).sample

最后，让我们将图像转换为 PIL 格式以便显示。

In [ ]:
image = (image / 2 + 0.5).clamp(0, 1)
image = image[0].detach().cpu().permute(1, 2, 0).numpy()
image = (image * 255).round().astype("uint8")
Image.fromarray(image)

### 仅代码版本

In [ ]:
prompts = [
    'a photograph of an astronaut riding a horse',
    'an oil painting of an astronaut riding a horse in the style of grant wood'
]

In [ ]:
text_input = tokenizer(prompts, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
text_embeddings = text_encoder(text_input.input_ids.to("cuda"))[0].half()

In [ ]:
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer([""] * len(prompts), padding="max_length", max_length=max_length, return_tensors="pt")
uncond_embeddings = text_encoder(uncond_input.input_ids.to("cuda"))[0].half()
emb = torch.cat([uncond_embeddings, text_embeddings])

In [ ]:
torch.manual_seed(100)
g = guidance_scale

In [ ]:
latents = torch.randn((len(prompts), unet.in_channels, height//8, width//8))
scheduler.set_timesteps(num_inference_steps)
latents = latents.to("cuda").half() * scheduler.init_noise_sigma

In [ ]:
for i,ts in enumerate(tqdm(scheduler.timesteps)):
    inp = scheduler.scale_model_input(torch.cat([latents] * 2), ts)
    with torch.no_grad(): u,t = unet(inp, ts, encoder_hidden_states=emb).sample.chunk(2)
    pred = u + g*(t-u)
    latents = scheduler.step(pred, ts, latents).prev_sample

In [ ]:
with torch.no_grad(): image = vae.decode(1 / 0.18215 * latents).sample
res = (image / 2 + 0.5).clamp(0, 1)

In [ ]:
image = res[0].detach().cpu().permute(1, 2, 0).numpy()
image = (image * 255).round().astype("uint8")
Image.fromarray(image)

In [ ]:
image = res[1].detach().cpu().permute(1, 2, 0).numpy()
image = (image * 255).round().astype("uint8")
Image.fromarray(image)

### 封装成函数

In [ ]:
def text_enc(prompts, maxlen=None):
    if maxlen is None: maxlen = tokenizer.model_max_length
    inp = tokenizer(prompts, padding="max_length", max_length=maxlen, truncation=True, return_tensors="pt")
    return text_encoder(inp.input_ids.to("cuda"))[0].half()

def mk_img(t):
    image = (t/2+0.5).clamp(0,1).detach().cpu().permute(1, 2, 0).numpy()
    return Image.fromarray((image*255).round().astype("uint8"))

In [ ]:
def mk_samples(prompts, g=7.5, seed=100, steps=70):
    bs = len(prompts)
    text = text_enc(prompts)
    uncond = text_enc([""] * bs, text.shape[1])
    emb = torch.cat([uncond, text])
    if seed: torch.manual_seed(seed)

    latents = torch.randn((bs, unet.in_channels, height//8, width//8))
    scheduler.set_timesteps(steps)
    latents = latents.to("cuda").half() * scheduler.init_noise_sigma

    for i,ts in enumerate(tqdm(scheduler.timesteps)):
        inp = scheduler.scale_model_input(torch.cat([latents] * 2), ts)
        with torch.no_grad(): u,t = unet(inp, ts, encoder_hidden_states=emb).sample.chunk(2)
        pred = u + g*(t-u)
        latents = scheduler.step(pred, ts, latents).prev_sample

    with torch.no_grad(): return vae.decode(1 / 0.18215 * latents).sample

In [ ]:
images = mk_samples(prompts)

In [ ]:
from IPython.display import display

In [ ]:
for img in images: display(mk_img(img))